In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import os
os.chdir('/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/PHCellTrackSeg')
!ls

drive_data_management.ipynb  Mask_merge.ipynb  res_track.txt
im2RGB.py		     README.md	       stack2im.py
man_track.txt		     requirements.txt  utils


In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 36.6MB 99kB/s 
     |████████████████████████████████| 42.5MB 109kB/s 


In [4]:
import sys
from utils.build_processed_videos import get_accuracy_measures_videos, build_videos, jaccard_index, dice_coeff
import utils.build_processed_videos
import os
import SimpleITK as sitk
import numpy as np
from scipy.spatial.distance import directed_hausdorff
import sys
import numpy as np
import glob
import SimpleITK as sitk
import os
import sys
from skimage import measure
import skimage.morphology
import scipy.ndimage
import cv2
from scipy import ndimage


In [27]:
INPUTPATH = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/Seg_github/Semantic-Segmentation-Suite/constante_tiramisu/2980/'
OUTPUTPATH = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/clean_data/constante_tiramisu/2980/'
PATH2VIDEOS = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/clean_data/data4training/test/videos2im_relation.csv'

In [28]:
build_videos(INPUTPATH,OUTPUTPATH,PATH2VIDEOS)

Processing video 5000_6-15-14_1_xy004_255-354:
[----------------------------------------------------------------------------------------------------] 100.0%
Processing video Videos and corresponding protrusion analysis_IL-8_6-15-14_1_xy033_181-183:
[---] 100.0%
Processing video 10000_11-20-13_1003_xy015_002-004:
[---] 100.0%
Processing video 5000_7-1-14_001_xy001_240-242:
[---] 100.0%
Processing video 10000_11-20-13_1003_xy008_078-080:
[---] 100.0%
Processing video 5000_7-1-14_001_xy005_157-171:
[---------------] 100.0%
Processing video Videos and corresponding protrusion analysis_IL-6_6-1-14_1001_xy28_330-332:
[---] 100.0%
Processing video 10000_11-20-13_1003_xy018_267-281:
[---------------] 100.0%
Processing video Videos and corresponding protrusion analysis_IL-6_12-5-12_1_xy27_370-385:
[----------------] 100.0%
Processing video Videos and corresponding protrusion analysis_IL-6_12-5-12_1_xy30_090-104:
[---------------] 100.0%
Processing video Videos and corresponding protrusion analy

In [6]:
def build_videos(INPUTPATH,OUTPUTPATH,PATH2VIDEOS):
    '''
    INPUTPATH: path where the single frames are placed.
    OUTPUTPATH: pathe where the reconstructed videos will be stored.
    PATH2VIDEOS: path to a txt file in which each row contains the name of the video and the frame that belongs to that
                video:
        Labels row      Videos ; Frames
                        video 1; raw_001.tif\n
                        video 1; raw_002.tif\n
                        ...
                        video 1; raw_032.tif\n
                        video 2; raw_033.tif\n
    '''
    files = [x for x in open(PATH2VIDEOS, "r")]
    files = files[1:]  # First row contains labels
    file_relation = [[x.split(';')[0], x.split(';')[1][:-1]] for x in files]

    if not os.path.exists(OUTPUTPATH):
        os.makedirs(OUTPUTPATH)

    COUNT = 1
    while COUNT <= len(file_relation):
        # Get the name of the original videos and the number of frames that it contains
        file_name = file_relation[COUNT][0] # video name
        # Calculate how many frames you need to process (it is said in the name of the video)
        start_time, end_time = file_name.split('_')[-1].split('-')
        start_time = np.int(start_time)
        end_time = np.int(end_time)
        sys.stdout.write("\rProcessing video {0}:\n".format(file_name))

        for i in range(end_time-start_time+1):
            # Load the single frame
            frame_name = os.path.join(INPUTPATH, 'raw_{0:0>3}_pred.tif'.format(int(COUNT+i)))
            frame = sitk.ReadImage(frame_name)
            frame = sitk.GetArrayFromImage(frame)
            if i == 0:
                video = frame.reshape((1,frame.shape[0], frame.shape[1]))
            else:
                video = np.concatenate((video,frame.reshape((1,frame.shape[0], frame.shape[1]))), axis=0)
            progress = (i+1)/(end_time-start_time+1)
            text = "\r[{0}] {1}%".format("-" * (i+1) + " " * (end_time-start_time-i), progress * 100)
            sys.stdout.write(text)
            sys.stdout.flush()
        COUNT = COUNT + i + 1
        sitk.WriteImage(sitk.GetImageFromArray(video.astype(np.uint16)), os.path.join(OUTPUTPATH, file_name + '.tif'))
        sys.stdout.write("\n")  # this ends the progress bar
    print("All videos have been reconstructed")

In [29]:
INPUTPATH = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/Seg_github/Semantic-Segmentation-Suite/constante_tiramisu/2980/'
OUTPUTPATH = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/clean_data/data4cellsegmentation/constante_tiramisu/2980/'
PATH2VIDEOS = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/clean_data/data4training/test/videos2im_relation.csv'

In [30]:
def stack2im(PATH, END='_Segmentationim-label'):
    # path to the data directory which contains folders such as train, val or test
    # PATH = "./data/"
    # subfolder in which the input and ground truth are
    # mode = 'train/'

    # Old data
    # name of the folder containing the input videos
    PATH2DATA = PATH + "inputs/"
    # name of the folder containing ground truth
    PATH2GT = PATH + 'labels/'
    # name of the folder containing results
    PATH2RES = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/clean_data/constante_tiramisu/2980/'

    # Ground truth's name has a different ending. Write it so as to get the correct
    # name of the input file
    # END = "_Segmentation2im_Prot"
    # END = '_Segmentationim-label'

    # New data
    # directory in which new files should be saved
    PATH2OUTPUT = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/clean_data/data4cellsegmentation_dense_cosntante_2980/'
    if not os.path.exists(PATH2OUTPUT):
        os.makedirs(PATH2OUTPUT)

    # name of the new ground truth
    masks_names = 'man_seg'
    # name of the new input slice
    input_names = 'raw_'

    res_name = 'mask_'

    FILES = glob.glob(PATH2GT + '/*.tif')
    # COUNT = 1
    COUNT = 1
    fields = "Video; Frame; Sequence"
    with open(os.path.join(PATH2OUTPUT, 'videos2seq.csv'), 'w') as file_:
        file_.write(fields)
        file_.write("\n")

    for file in FILES:
        if not os.path.exists(PATH2OUTPUT+ str('/0')+ str(COUNT)):
            os.makedirs(PATH2OUTPUT+ str('/0')+ str(COUNT))
        if not os.path.exists(PATH2OUTPUT+str('/0')+ str(COUNT)+'_GT/SEG'):
            os.makedirs(PATH2OUTPUT+str('/0') + str(COUNT) + '_GT/SEG')
        if not os.path.exists(PATH2OUTPUT+str('/0')+ str(COUNT)+'_RES'):
             os.makedirs(PATH2OUTPUT+str('/0') + str(COUNT) + '_RES')
        # Load masks from segmented videos
        file_name = file
        masks = sitk.ReadImage(file_name)
        masks = sitk.GetArrayFromImage(masks)

        # Load original images corresponding to masks.
        file_name = file_name.split('/')[-1].split('.')[0]

        # Option 1 for the input names
        time_range = file_name[:-len(END)].split('_')[-1]
        image_name = file_name[:-(len(END + time_range))] + 'stackreg_' + time_range + '.tif'
        res_name = file_name[:-(len(END + time_range))] + time_range + '.tif'

        # Option 2 for the input names
        # image_name = file_name[:-(len(END))] + '.tif'

        image = sitk.ReadImage(PATH2DATA + image_name)
        image = sitk.GetArrayFromImage(image)

        res = sitk.ReadImage(PATH2RES + res_name)
        res = sitk.GetArrayFromImage(res)

        for i in range(masks.shape[0]):
            aux = masks[i]
            aux = aux.astype(np.uint16)
            aux[aux>0.5]=1
            number = np.str(i)
            sitk.WriteImage(sitk.GetImageFromArray(aux),
                            PATH2OUTPUT + str('/0') + str(COUNT) +'_GT/SEG/' + str('man_seg0')+'{0:0>3}'.format(int(i)) + '.tif')
            print('File {} number {} processed.'.format(masks_names, COUNT))

            aux = image[i]
            aux = aux.astype(np.uint16)
            number = np.str(i)
            sitk.WriteImage(sitk.GetImageFromArray(aux),
                            PATH2OUTPUT + str('/0') + str(COUNT) +'/' + str('0') +'{0:0>3}'.format(int(i)) + '.tif')
            
            aux = res[i]
            aux, _ = ndimage.measurements.label(aux)
            aux = aux.astype(np.uint16)
            aux[aux>0.5]=1
            number = np.str(i)
            sitk.WriteImage(sitk.GetImageFromArray(aux),
                            PATH2OUTPUT + str('/0') + str(COUNT) +'_RES/' + str('mask0')+'{0:0>3}'.format(int(i)) + '.tif')
            print('File {} number {} processed.'.format(masks_names, COUNT))

            with open(os.path.join(PATH2OUTPUT, 'videos2seq.csv'), 'a') as file_:
                file_.write('{};'.format(file_name[:-len(END)]) + input_names + '{0:0>3}.tif'.format(int(i)))
                file_.write("\n")
            print('File {} number {} processed.'.format(input_names, COUNT))

        COUNT +=1
        print('File {} converted.'.format(file_name))
    print('Process finished')

In [31]:
stack2im('/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/clean_data/data4training/test/')

File man_seg number 1 processed.
File man_seg number 1 processed.
File raw_ number 1 processed.
File man_seg number 1 processed.
File man_seg number 1 processed.
File raw_ number 1 processed.
File man_seg number 1 processed.
File man_seg number 1 processed.
File raw_ number 1 processed.
File 10000_11-20-13_1003_xy008_078-080_Segmentationim-label converted.
File man_seg number 2 processed.
File man_seg number 2 processed.
File raw_ number 2 processed.
File man_seg number 2 processed.
File man_seg number 2 processed.
File raw_ number 2 processed.
File man_seg number 2 processed.
File man_seg number 2 processed.
File raw_ number 2 processed.
File man_seg number 2 processed.
File man_seg number 2 processed.
File raw_ number 2 processed.
File man_seg number 2 processed.
File man_seg number 2 processed.
File raw_ number 2 processed.
File man_seg number 2 processed.
File man_seg number 2 processed.
File raw_ number 2 processed.
File man_seg number 2 processed.
File man_seg number 2 processed.

In [32]:
def stack2im_tra(PATH, END='_Segmentationim-label'):
    # path to the data directory which contains folders such as train, val or test
    # PATH = "./data/"
    # subfolder in which the input and ground truth are
    # mode = 'train/'

    # Old data
    # name of the folder containing the input videos
    PATH2DATA = PATH + "inputs/"
    # name of the folder containing ground truth
    PATH2GT = PATH + 'labels/'
    # name of the folder containing results
    PATH2RES = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/clean_data/constante_tiramisu/2980/'

    # Ground truth's name has a different ending. Write it so as to get the correct
    # name of the input file
    # END = "_Segmentation2im_Prot"
    # END = '_Segmentationim-label'

    # New data
    # directory in which new files should be saved
    PATH2OUTPUT = '/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/clean_data/data4cellsegmentation_dense_constante_tra_2980/'
    if not os.path.exists(PATH2OUTPUT):
        os.makedirs(PATH2OUTPUT)
        print('done')

    # name of the new ground truth
    masks_names = 'man_track'
    # name of the new input slice
    input_names = 'raw_'

    res_name = 'mask'

    FILES = glob.glob(PATH2GT + '/*.tif')
    COUNT = 1
    fields = "Video; Frame; Sequence"
    with open(os.path.join(PATH2OUTPUT, 'videos2seq.csv'), 'w') as file_:
        file_.write(fields)
        file_.write("\n")


    for file in FILES:
        if not os.path.exists(PATH2OUTPUT+ str('/0')+ str(COUNT)):
            os.makedirs(PATH2OUTPUT+ str('/0')+ str(COUNT))
        if not os.path.exists(PATH2OUTPUT+str('/0')+ str(COUNT)+'_GT/TRA'):
            os.makedirs(PATH2OUTPUT+str('/0') + str(COUNT) + '_GT/TRA')
        if not os.path.exists(PATH2OUTPUT+str('/0')+ str(COUNT)+'_RES'):
             os.makedirs(PATH2OUTPUT+str('/0') + str(COUNT) + '_RES')
        # Load masks from segmented videos
        file_name = file
        masks = sitk.ReadImage(file_name)
        masks = sitk.GetArrayFromImage(masks)

        # Load original images corresponding to masks.
        file_name = file_name.split('/')[-1].split('.')[0]

        # Option 1 for the input names
        time_range = file_name[:-len(END)].split('_')[-1]
        image_name = file_name[:-(len(END + time_range))] + 'stackreg_' + time_range + '.tif'
        res_name = file_name[:-(len(END + time_range))] + time_range + '.tif'

        # Option 2 for the input names
        # image_name = file_name[:-(len(END))] + '.tif'

        image = sitk.ReadImage(PATH2DATA + image_name)
        image = sitk.GetArrayFromImage(image)

        res = sitk.ReadImage(PATH2RES + res_name)
        res = sitk.GetArrayFromImage(res)

      
        for i in range(masks.shape[0]):
            aux = masks[i]
            aux = aux.astype(np.uint16)
            number = np.str(i)
            sitk.WriteImage(sitk.GetImageFromArray(aux),
                            PATH2OUTPUT + str('/0') + str(COUNT) +'_GT/TRA/' + str('man_track0')+'{0:0>3}'.format(int(i)) + '.tif')
            print('File {} number {} processed.'.format(masks_names, COUNT))

            aux = image[i]
            aux = aux.astype(np.uint16)
            number = np.str(i)
            sitk.WriteImage(sitk.GetImageFromArray(aux),
                            PATH2OUTPUT + str('/0') + str(COUNT) +'/' + str('0') +'{0:0>3}'.format(int(i)) + '.tif')
            
            aux = res[i]
            aux, _ = ndimage.measurements.label(aux)
            aux = aux.astype(np.uint16)
            number = np.str(i)
            sitk.WriteImage(sitk.GetImageFromArray(aux),
                            PATH2OUTPUT + str('/0') + str(COUNT) +'_RES/' + str('mask0')+'{0:0>3}'.format(int(i)) + '.tif')
            print('File {} number {} processed.'.format(masks_names, COUNT))

            with open(os.path.join(PATH2OUTPUT, 'videos2seq.csv'), 'a') as file_:
                file_.write('{};'.format(file_name[:-len(END)]) + input_names + '{0:0>3}.tif'.format(int(i)))
                file_.write("\n")
            print('File {} number {} processed.'.format(input_names, COUNT))

        COUNT +=1
        print('File {} converted.'.format(file_name))
    print('Process finished')

In [33]:
stack2im_tra('/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/clean_data/data4training/test/')

done
File man_track number 1 processed.
File man_track number 1 processed.
File raw_ number 1 processed.
File man_track number 1 processed.
File man_track number 1 processed.
File raw_ number 1 processed.
File man_track number 1 processed.
File man_track number 1 processed.
File raw_ number 1 processed.
File 10000_11-20-13_1003_xy008_078-080_Segmentationim-label converted.
File man_track number 2 processed.
File man_track number 2 processed.
File raw_ number 2 processed.
File man_track number 2 processed.
File man_track number 2 processed.
File raw_ number 2 processed.
File man_track number 2 processed.
File man_track number 2 processed.
File raw_ number 2 processed.
File man_track number 2 processed.
File man_track number 2 processed.
File raw_ number 2 processed.
File man_track number 2 processed.
File man_track number 2 processed.
File raw_ number 2 processed.
File man_track number 2 processed.
File man_track number 2 processed.
File raw_ number 2 processed.
File man_track number 2 

In [34]:
def change(path):
  sheet_name='man_track.txt'
  sheet_name_re = 'res_track.txt'
  for i in range(16):
      print(i+1)
      pathtra= path + str('/0') + str(i+1) +'_GT/TRA/'
      maxi=0
      frame=0
      FILES = glob.glob(pathtra + '/*.tif')
      for file in FILES:
          file_name = file
          file_name = file_name.split('/')[-1].split('.')[0]
          masks = sitk.ReadImage(file)
          masks = sitk.GetArrayFromImage(masks)
          masks = masks.astype(np.uint16)
          masks=masks+maxi
          masks[masks==masks.min()]=0
          uni = np.unique(masks)
          maxi = max(np.unique(masks))
          sitk.WriteImage(sitk.GetImageFromArray(masks), pathtra + file_name + '.tif')
          if os.path.exists(pathtra + sheet_name) == 0:   
                with open(pathtra + sheet_name, 'w') as file_:
                    for j in range(len(uni)-1):
                          wri = str((uni[j+1], frame,  frame, 0))
                          file_.write(wri)
                          file_.write("\n")
          else:
            with open(pathtra + sheet_name, 'a') as file_:
              for k in range(len(uni)-1):
                wri = str((uni[k+1], frame,  frame, 0))
                file_.write(wri)
                file_.write("\n")
          frame +=1
      file_.close()
      path2mask = path + str('/0') + str(i+1) +'_RES/'
      maxi=0
      frame=0
      FILES = glob.glob(path2mask + '/*.tif')
      for file in FILES:
          file_name = file
          file_name = file_name.split('/')[-1].split('.')[0]
          masks = sitk.ReadImage(file)
          masks = sitk.GetArrayFromImage(masks)
          masks = masks.astype(np.uint16)
          masks=masks+maxi
          masks[masks==masks.min()]=0
          uni = np.unique(masks)
          maxi = max(np.unique(masks))
          sitk.WriteImage(sitk.GetImageFromArray(masks), path2mask + file_name + '.tif')
          if os.path.exists(path2mask + sheet_name_re) == 0:   
                with open(path2mask + sheet_name_re, 'w') as file_mas:
                  for m in range(len(uni)-1):
                    wri = str((uni[m+1], frame,  frame, 0))
                    file_mas.write(wri)
                    file_mas.write("\n")
          else:
            with open(path2mask + sheet_name_re, 'a') as file_mas:
                for l in range(len(uni)-1):
                  wri = str((uni[l+1], frame,  frame, 0))
                  file_mas.write(wri)
                  file_mas.write("\n")
          frame +=1
        # file_mas.close()


In [36]:
change('/content/gdrive/My Drive/TFG/TFG MARINA CALZADA/clean_data/data4cellsegmentation_dense_constante_tra_2980/')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
